In [551]:
import pandas as pd
import numpy as np

import os

In [552]:
# Log file from psychopy.
log_fn=os.path.join('..',
                    'data', 
                    'logfiles', 
                    'sub-01_brainplayback_quadrants_2022-11-23_15h00.32.477_run-1.csv')



In [553]:
#create a new lists by parsing the string using "," delimiter

head_tail = os.path.split(log_fn)
print(head_tail[1])

sub_id=head_tail[1].split("_")

print("Participant: ",sub_id[0])

run_id=sub_id[-1].split(".")
print("Run id: ",run_id[0])

sub-11_BrainplaybackQuadrants_2023-03-08_18h14.55.471_run-4.csv
Participant:  sub-11
Run id:  run-4


In [554]:
# run data
sub_id=sub_id[0]
run_id=run_id[0]

# default values
sess_id='01'
task_id='01'


In [555]:
# Create pandas dataframe to read data into data structure.
df_log=pd.read_csv(log_fn)


In [556]:
df_log.columns

Index(['MusicsSelect', 'musics', 'trials.thisRepN', 'trials.thisTrialN',
       'trials.thisN', 'trials.thisIndex', 'blocks.thisRepN',
       'blocks.thisTrialN', 'blocks.thisN', 'blocks.thisIndex',
       'trialsMusics.thisRepN', 'trialsMusics.thisTrialN',
       'trialsMusics.thisN', 'trialsMusics.thisIndex',
       'WelcomeMessage.started', 'WelcomeMessage.stopped', 'Baseline1.started',
       'Noise.started', 'Baseline1.stopped', 'Baseline2.started',
       'Baseline2.stopped', 'MusicExcerpt.started', 'ISI.started',
       'MusicExcerpt.stopped', 'ISI.stopped', 'Noise.stopped',
       'Baseline3.started', 'GoodbyeText.started', 'Baseline3.stopped',
       'GoodbyeText.stopped', 'participant', 'session', 'trigger', 'date',
       'expName', 'psychopyVersion', 'frameRate', 'Unnamed: 37'],
      dtype='object')

In [557]:
# Select subset of columns
df_log_Q=df_log.loc[:,["MusicsSelect","Baseline1.started", "Noise.started","Baseline2.started", "MusicExcerpt.started"]]

# Remove rows with NaN - nor necessary for this event files.
df_log_Q=df_log_Q.dropna(how="any")


In [558]:
df_log_Q

,MusicsSelect,Baseline1.started,Noise.started,Baseline2.started,MusicExcerpt.started
0,MusicListQ2.csv,45.780532,57.738358,69.725384,81.759662
4,MusicListQ1.csv,117.734543,129.727585,141.736018,153.778573
8,MusicListQ4.csv,189.740659,201.730552,213.736332,225.777944
12,MusicListQ3.csv,261.745156,273.732757,285.736367,297.783808
17,MusicListQ1.csv,333.780528,345.768862,357.765772,369.786664
21,MusicListQ4.csv,405.792975,417.771665,429.776147,441.823115
25,MusicListQ3.csv,477.796855,489.774646,501.776264,513.825288
29,MusicListQ2.csv,549.802713,561.776792,573.776461,585.810849


In [559]:
# Select subset of columns
df_last_bl=df_log.loc[:,["Baseline3.started", "Baseline3.stopped"]]

# Remove rows with NaN - nor necessary for this event files.
df_last_bl=df_last_bl.dropna(how="any")


In [560]:
df_last_bl

,Baseline3.started,Baseline3.stopped
34,621.88305,645.892874


In [561]:
# Create pandas dataframe with columns required by the BIDS standard.
bids={'onset':[],'duration':[],'trial_type':[]}
df_bids=pd.DataFrame(bids)

In [562]:
#number of decimals when rounding numbers
decimals=2

# offset of the experiment in seconds (seconds until first baseline.)
exp_offset=df_log_Q.iloc[0]["Baseline1.started"]

# considering the log file, add rows for each event.

# For each row:
for i in range(len(df_log_Q)-1):
    # Add Baseline trial.
    df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["Baseline1.started"]-exp_offset, decimals),
                                       np.round(df_log_Q.iloc[i]["Noise.started"]-df_log_Q.iloc[i]["Baseline1.started"],
                                                decimals),
                                      'Baseline'] 
    # Add Noise trial.
    df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["Noise.started"]-exp_offset, decimals),
                                       np.round(df_log_Q.iloc[i]["Baseline2.started"]-df_log_Q.iloc[i]["Noise.started"],
                                                decimals),
                                      'Noise'] 
    # Add Baseline2 trial.
    df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["Baseline2.started"]-exp_offset, decimals),
                                       np.round(df_log_Q.iloc[i]["MusicExcerpt.started"]-df_log_Q.iloc[i]["Baseline2.started"],
                                                decimals),
                                      'Baseline'] 
    
    # Add Music trial.
    df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["MusicExcerpt.started"]-exp_offset, decimals),
                                       np.round(df_log_Q.iloc[i+1]["Baseline1.started"]-df_log_Q.iloc[i]["MusicExcerpt.started"], decimals),
                                       df_log_Q.iloc[i]["MusicsSelect"][-6:-4]] 
    
# Last block.
i=i+1

# Add Baseline trial.
df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["Baseline1.started"]-exp_offset, decimals),
                                   np.round(df_log_Q.iloc[i]["Noise.started"]-df_log_Q.iloc[i]["Baseline1.started"],
                                            decimals),
                                      'Baseline'] 
# Add Noise trial.
df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["Noise.started"]-exp_offset, decimals),
                                   np.round(df_log_Q.iloc[i]["Baseline2.started"]-df_log_Q.iloc[i]["Noise.started"],
                                            decimals),
                                  'Noise'] 
# Add Baseline2 trial.
df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["Baseline2.started"]-exp_offset, decimals),
                                   np.round(df_log_Q.iloc[i]["MusicExcerpt.started"]-df_log_Q.iloc[i]["Baseline2.started"],
                                            decimals),
                                  'Baseline'] 

# Add Music trial.
df_bids.loc[len(df_bids.index)] = [np.round(df_log_Q.iloc[i]["MusicExcerpt.started"]-exp_offset, decimals),
                                   np.round(df_last_bl.iloc[0]["Baseline3.started"]-df_log_Q.iloc[i]["MusicExcerpt.started"], decimals),
                                   df_log_Q.iloc[i]["MusicsSelect"][-6:-4]] 


df_bids.loc[len(df_bids.index)] = [np.round(df_last_bl.iloc[0]["Baseline3.started"]-exp_offset, decimals),
                                       np.round(df_last_bl.iloc[0]["Baseline3.stopped"]-df_last_bl.iloc[0]["Baseline3.started"], decimals),
                                       'Baseline'] 


    

In [563]:
df_bids

,onset,duration,trial_type
0,0.00,11.96,Baseline
1,11.96,11.99,Noise
2,23.94,12.03,Baseline
3,35.98,35.97,Q2
4,71.95,11.99,Baseline
5,83.95,12.01,Noise
6,95.96,12.04,Baseline
7,108.00,35.96,Q1
8,143.96,11.99,Baseline
9,155.95,12.01,Noise


In [564]:
# Create tsv file.
fn=f"{sub_id}_ses-{sess_id}_task-{task_id}_{run_id}_events.tsv"
df_bids.to_csv(os.path.join('bids-events-files',fn), sep="\t", index=False)

In [565]:
del df_bids